<a href="https://colab.research.google.com/github/lizardnote/Text-Analytics/blob/main/RAG_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앞으로 뭘하나요?  

1. PDF 파일을 다운로드 합니다.
2. Langchain의 PyPDFLoader로 PDF 파일을 읽어서 페이지 별로 저장합니다.
  - 예를 들어 27페이지의 PDF 파일이라면 27개의 문서를 얻게 됩니다.  
3. 27개의 문서들은 길이가 제각각입니다. 특정 길이(여기서는 500)로 제한하고 더 잘라줍니다.
4. 총 69개의 문서가 나오게 됩니다. 69개의 문서를 전부 임베딩(Embedding)하여 벡터 데이터베이스에 적재합니다.  
5. 벡터 데이터베이스는 Chroma 또는 Faiss를 사용할 것입니다.  
6. 질문을 입력하면 69개의 문서 중 가장 유사도가 높은 문서 4개를 출력합니다.

## 필요한 패키지 다운로드

필요한 패키지를 다운로드 합니다.  

랭체인, 벡터데이터베이스, 임베딩 모델, PDF 파일을 읽는 도구들입니다.

In [2]:
!pip install langchain langchain_openai chromadb transformers sentence-transformers pypdf langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 120.8 MB/s eta

## Langchain PDF 로드 및 페이지 별 분할

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  #text split 단계
from langchain.vectorstores import Chroma                           #vector database
from langchain.document_loaders import PyPDFLoader                  #PDF loader
from langchain.embeddings import HuggingFaceEmbeddings              #embedding

삼성전자 기업분석 PDF를 다운로드합니다.

In [2]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

--2025-05-30 09:20:07--  https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
Resolving wdr.ubion.co.kr (wdr.ubion.co.kr)... 61.100.182.43
Connecting to wdr.ubion.co.kr (wdr.ubion.co.kr)|61.100.182.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253369 (1.2M) [application/pdf]
Saving to: ‘gsat_170823.pdf’

gsat_170823.pdf     100%[===================>]   1.19M   951KB/s    in 1.3s    

2025-05-30 09:20:10 (951 KB/s) - ‘gsat_170823.pdf’ saved [1253369/1253369]



## PDF 페이지 로드

PDF를 Langchain의 PyPDFLoader로 로드 후 분할(총 27P)  
따라서 문서를 읽고 분할하면 총 27개의 문서가 나온다.

In [4]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

27


In [5]:
print(len(pages))

27


첫번째 문서를 출력해봅시다. 랭체인으로 로드한 데이터의 형식은 다음을 따릅니다.  

`Document(metadata={'source': 파일명, 'page': 페이지 번호}, page_content='내용')`

In [6]:
pages[0]

Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품

## 청킹(Chunking): 문서 분할

27개의 문서는 길이가 제각각입니다. 길이 500기준으로 추가적으로 더 쪼개준다.  
27개의 문서 중 어떤 문서가 길이가 800이라면 이 문서는 두 개로 다시 잘라서 500이 넘지 않도록 하는 것

추가적으로 문서들을 길이 별로 자르는 것은 `Langchain의 RecursiveCharacterTextSplitter`를 통해서 가능

문서가 69개로 쪼개진다.

In [7]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splited_docs = text_splitter.split_documents(pages)

print(len(splited_docs))

69


이제 문서는 69개가 되었습니다.

In [8]:
print('69개의 문서 중 첫번째 문서:', splited_docs[0])

69개의 문서 중 첫번째 문서: page_content='2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산1) 244조 매출액2) 200조
임직원수 95,374명 
∙ 1975년 1월 주식시장 상장
∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 
∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) 
3개의 부문으로 나누어 독립 경영.
부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}


실제로 길이를 재보면 500이 넘지 않는 것을 확인할 수 있습니다.

In [9]:
print('첫번째 문서의 길이:', len(splited_docs[0].page_content))

첫번째 문서의 길이: 490


69개의 문서를 임베딩해야 합니다. 한글이 가능한 공개되어져 있는 오픈소스 임베딩을 사용해보겠습니다.  

임베딩 모델 위치: https://huggingface.co/BAAI/bge-m3

In [10]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

<ipython-input-10-029e6670ad57>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## 크로마(Chroma)

다음은 Langchain에서 Chroma를 이용하여 벡터 DB 사용(FAISS 등 다른 DB도 사용 가능)


```python
# Chroma 를 통해 벡터 저장소 생성. OpenAIEmbeddings()를 사용하거나 HuggingFaceEmbeddings()를 사용하면 된다.
chroma_db = Chroma.from_documents(splited_docs, HuggingFaceEmbeddings())

# 유사도 검색(쿼리)
similar_docs = chroma_db.similarity_search("사용자의 질문")
```

실제로 실행해봅시다.

In [11]:
## Chroma 기반 pdf(docs 벡터화)
db = Chroma.from_documents(splited_docs, model_huggingface)

print('문서의 수:', db._collection.count())

문서의 수: 69


69개의 문서가 위에서 로드한 오픈 소스 임베딩을 바탕으로 임베딩된 단계이다.

임의로 '삼성전자의 주요 사업영역은?' 이라는 질문을 던지고 임베딩 벡터의 유사도 기준으로 상위 4개의 문서를 출력해보면 아래와 같다

In [12]:
## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db.similarity_search(question, k=4)

In [13]:
print('검색된 문서의 수:', len(docs))

검색된 문서의 수: 4


In [14]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'page': 9, 'creator': 'nPDF (pdftk 1.41)', 'page_label': '10', 'moddate': '2017-08-16T00:21:02-08:00', 'creationdate': '2017-08-16T00:21:02-08:00', 'total_pages': 27, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)'}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 벡터 데이터베이스 로컬에 파일로 저장

vectorize된 텍스트를 Chroma db 파일로 저장하기

In [ ]:
db_toFiles = Chroma.from_documents(splited_docs, model_huggingface, persist_directory = './samsumg.db')
print('문서의 수:', db_toFiles._collection.count())

문서의 수: 69


이 시점에 현재 경로에 samsung.db라는 파일이 생긴 것을 볼 수 있고 다시 불러와서 활용할 수도 있다.

In [ ]:
# 현재 경로에 있는 파일들 확인
%ls -al

total 1244
drwxr-xr-x 1 root root    4096 May 29 07:40 ./
drwxr-xr-x 1 root root    4096 May 29 07:35 ../
drwxr-xr-x 4 root root    4096 May 23 13:38 .config/
-rw-r--r-- 1 root root 1253369 Aug 23  2017 gsat_170823.pdf
drwxr-xr-x 1 root root    4096 May 23 13:39 sample_data/
drwxr-xr-x 3 root root    4096 May 29 07:40 samsumg.db/


In [15]:
db_fromfile = Chroma(persist_directory = './samsumg.db',embedding_function=model_huggingface)
print('문서의 수:', db_fromfile._collection.count())

## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db_fromfile.similarity_search(question, k=4)

print('검색된 문서의 수:', len(docs))

문서의 수: 0
검색된 문서의 수: 0


<ipython-input-15-321936a76263>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_fromfile = Chroma(persist_directory = './samsumg.db',embedding_function=model_huggingface)


유사도 점수를 함께 출력하거나 상위 문서의 개수를 조절

In [16]:
question = '삼성전자의 주요 사업영역은?'

# TOP 3개만 추출
docs3 = db_fromfile.similarity_search_with_relevance_scores(question, k=3)

In [17]:
for doc in docs3:
  print(doc)
  print('--' * 100)

위의 검색 결과의 맨 오른쪽을 보면 유사도 점수가 측정되어 출력된다.

**주의사항**  
이 유사도 점수를 일종의 임계값으로 사용하는 것은 정확도가 보장되는 방법은 아님.

이 임베딩 모델은 여러분들이 실습하고 있는 도메인 데이터에 특화된 임베딩 모델이 아니기 때문이다.
임베딩 모델을 해당 도메인으로 파인 튜닝 했을 때 의미가 있다.

유사도가 가장 높은 문서와 유사도 점수 출력

In [ ]:
print(f"가장 유사한 문서 : {docs3[0][0].page_content}")
print('*'*20)
print(f"가장 유사한 문서의 문서의 유사도 : {docs3[0][1]}")

가장 유사한 문서 : 11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
********************
가장 유사한 문서의 문서의 유사도 : 0.6830467219193441


## GPT-4 호출하기

OpenAI key 값을 입력하여 GPT-4 사용

In [18]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import os

In [24]:
os.environ['OPENAI_API_KEY'] = ""

이제 GPT-4에 전달할 프롬프트 작성

In [25]:
# GPT-4에 전달할 프롬프트
template = """당신은 삼성전자 기업 보고서를 설명해주는 챗봇 '삼성맨'입니다.
안상준 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 이모지를 사용하며 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

# 프롬프트 템플릿은 {question}에는 사용자의 질문이 들어간다. {context}에는 사용자의 질문에 따른 검색 결과가 들어간다.
prompt = PromptTemplate.from_template(template)

In [26]:
# GPT-4 선언. 단, API Key 값이 셋팅되어져 있어야 실행 가능합니다.
llm = ChatOpenAI(model_name="gpt-4o")

ChatOpenAI로부터 llm 객체를 선언하여 GPT-4 호출

In [28]:
result = llm.invoke('안녕 반가워 GPT-4야 나는 효재야')
print(result.content)

안녕하세요 효재님! 만나서 반가워요. 어떻게 도와드릴까요?


## 벡터 데이터베이스와 LLM의 연결

`RetrievalQA.from_chain_type()`

- llm: 앞서 선언한 llm 객체를 넣고
- retriever: 앞서 선언한 retriever 객체를 넣어준다.
- chain_type_kwargs={"prompt": ?}: 앞서 선언한 프롬프트 템플릿을 넣어준다.

In [29]:
# 벡터 데이터베이스를 LLM과 연결하기 위해서 retriever 객체로 선언
retriever = db.as_retriever(search_kwargs={"k": 3})

In [30]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, # LLM 연결
    retriever=retriever, # 리트리버(벡터 데이터베이스) 연결
    chain_type_kwargs={"prompt": prompt}, # 프롬프트 템플릿 연결
    return_source_documents=True # 실제 검색된 문서도 확인하려면 True
    )

In [31]:
input_text = "삼성전자의 주요 사업영역은?"
chatbot_response = qa_chain.invoke(input_text)

In [32]:
chatbot_response

{'query': '삼성전자의 주요 사업영역은?',
 'result': '안녕하세요! 😊 삼성전자의 주요 사업영역에 대해 설명해드리겠습니다. 삼성전자는 크게 세 개의 독립 경영 부문으로 나누어져 있습니다.\n\n1. **Consumer Electronics (CE) 부문**:\n   - 영상디스플레이: 스마트TV와 초대형 프리미엄 TV 전략\n   - 생활가전: 새로운 기술과 가치를 창출\n   - 의료기기: 혁신적인 진단 기기 개발\n\n2. **Information technology & Mobile communication (IM) 부문**:\n   - 무선: 소비자들이 열망하는 새로운 가치와 편의를 제공\n\n3. **Device Solutions (DS) 부문**:\n   - 반도체, 메모리 등의 솔루션 제공\n\n이 부문들은 각각 다양한 제품과 서비스를 제공하며 독립적으로 경영되고 있습니다. 📺📱💾',
 'source_documents': [Document(metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'creator': 'nPDF (pdftk 1.41)', 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'creationdate': '2017-08-16T00:21:02-08:00'}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(D

In [33]:
print('내가 넣었던 질문:', chatbot_response['query'])

내가 넣었던 질문: 삼성전자의 주요 사업영역은?


In [34]:
print('검색 결과를 바탕으로 GPT-4가 작성한 답변:', chatbot_response['result'])

검색 결과를 바탕으로 GPT-4가 작성한 답변: 안녕하세요! 😊 삼성전자의 주요 사업영역에 대해 설명해드리겠습니다. 삼성전자는 크게 세 개의 독립 경영 부문으로 나누어져 있습니다.

1. **Consumer Electronics (CE) 부문**:
   - 영상디스플레이: 스마트TV와 초대형 프리미엄 TV 전략
   - 생활가전: 새로운 기술과 가치를 창출
   - 의료기기: 혁신적인 진단 기기 개발

2. **Information technology & Mobile communication (IM) 부문**:
   - 무선: 소비자들이 열망하는 새로운 가치와 편의를 제공

3. **Device Solutions (DS) 부문**:
   - 반도체, 메모리 등의 솔루션 제공

이 부문들은 각각 다양한 제품과 서비스를 제공하며 독립적으로 경영되고 있습니다. 📺📱💾


In [35]:
print('챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:')

for doc in chatbot_response['source_documents']:
  print(doc)
  print('--' * 100)

챗봇이 참고한 실제 검색 결과에 해당하는 유사도 상위 3개 문서:
page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'creator': 'nPDF (pdftk 1.41)', 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'creationdate': '2017-08-16T00:21:02-08:00'}
-------------------------------------------------------------------------------------------------------------------------------

In [36]:
input_text = "25년 2월의 서울 아파트 집값은?"
chatbot_response = qa_chain.invoke(input_text)

In [37]:
chatbot_response['result']

'죄송하지만 제공된 검색 결과에는 서울 아파트 집값에 대한 정보가 없습니다. 다른 질문 있으시면 말씀해 주세요! 😊'

프롬프트에 맞게 알 수 없는 정보는 모른다고 대답하는 것을 확인

## 성능 향상을 위한 고민 포인트

시간의 한계로 인해 여기서 자세히 다루지는 못하지만 여러분들이 앞으로 공부하는 방향에 있어서 키워드들을 제시하고자 합니다.

**Q. PDF를 페이지 별로 로드하고 또 그 페이지들을 적당한 길이로 자르면 각각의 문서들은 내용이 앞이나 뒤에 문맥이 없이 중간에 끊길텐데 이대로 검색 문서에 적재되어도 괜찮을까요?**

A) 당연히 성능에 영향을 줍니다. Chunking에 대한 고민을 많이 해야 하고 이를 위한 다양한 검색 알고리즘과 청킹 알고리즘들이 존재하여 이에 대해서 공부하고 적용해야 합니다.  

아래 질문과 연계된 질문이기도 하므로 아래 질문과 맞추어서 보충 답변을 해보겠습니다.

**Q. 사용자의 질문에 대한 답이 검색된 문서에 없을 경우에는 어떻게 해야할까요?**

A) RAG에서는 기본적으로 검색된 문서에 대한 답이 없을 때에 대한 시나리오를 따로 준비해야 합니다. 여기서는 검색 결과가 없다라고 단순하게 답변하도록 유도하였습니다.  

실제로 질문이 갖고 있는 문서로 답변할 수 없는 경우인지 원래 검색이 되어야 하는데 검색 성능이 안 좋아서 검색이 안 된 건지를 알아야 합니다. 만약 후자의 경우라면 검색 성능을 높이기 위해서 다음과 같은 고민들을 해야 합니다.  

- 문서 Chunking을 잘못해서 임베딩 성능이 좋지 않지는 않은가?
  - semantic chunking, 합성 데이터 방식(요약 등), 앤트로픽의 Contextual Retrieval을 고민해볼까?
- 임베딩 모델의 성능이 좋지 않아서 파인 튜닝을 해볼까?  
- reranking이라고 하는 2단계 검색 과정을 넣어서 검색 성능을 높여야 할까?  
- BM25와 같은 전통적인 키워드 기반의 검색 알고리즘을 혼합해볼까?  
- HyDE와 같은 검색 성능을 높이는 방법을 적용해볼까?  



**Q. PDF 파일에 이미지나 테이블이 있었을텐데 이런 건 제대로 검색이 되고 또 답변할 수 있을까요?**

A) 멀티모달 RAG의 영역입니다. VLM과 같은 멀티모달 LLM, 멀티모달 임베딩, 또는 OCR이 가능한 패키지인 unstructured.io 등을 사용하여 다양한 방법으로 이를 처리할 수 있습니다.

**Q. 멀티턴, 멀티쿼리 등과 같은 복잡한 쿼리에 대해서도 대응 가능할까?**

A) 멀티턴은 현 구조에서도 조금의 수정을 통해 가능하지만, 멀티 쿼리(A and B)는 현 구조에서는 어렵습니다. 이 경우 에이전트 RAG를 수행하거나 앞단에 별도의 모듈을 두어서 처리해야 합니다.